In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/200k-short-texts-for-humor-detection/dataset.csv
/kaggle/input/200k-short-texts-for-humor-detection/test.csv
/kaggle/input/200k-short-texts-for-humor-detection/train.csv
/kaggle/input/200k-short-texts-for-humor-detection/dev.csv


In [2]:
!ls

__notebook__.ipynb


In [3]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())

print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

print('# RAM')
run('cat /proc/meminfo | egrep "^MemTotal"')

print('# GPU')
run('lspci | grep VGA')

print('# OS')
run('uname -a')

# CPU
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
cpu MHz		: 2000.170
cpu cores	: 1
# RAM
MemTotal:       16430196 kB
# GPU

# OS
Linux 2e3392768f92 4.19.112+ #1 SMP Tue Jul 28 14:15:29 PDT 2020 x86_64 x86_64 x86_64 GNU/Linux


In [4]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *

import seaborn as sns
import string
import re    #for regex

np.set_printoptions(suppress=True)
print(tf.__version__)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


2.2.0


In [5]:
from transformers import BertTokenizer, BertModel


MODEL_TYPE = 'bert-base-uncased'
MAX_SIZE = 200
BATCH_SIZE = 200

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)

In [6]:
HAS_ANS = False
training_sample_count = 1200 # 4000
training_epochs = 3 # 3
test_count = 20000
# training_sample_count = 1000 # 4000
# training_epochs = 1 # 3
# test_count = 1000

running_folds = 1 # 2
MAX_SEQUENCE_LENGTH = 256

In [7]:
!ls /kaggle/input/200k-short-texts-for-humor-detection

dataset.csv  dev.csv  test.csv	train.csv


In [8]:
!ls /kaggle/input/cbert-after-preprocessing-by-1

ls: cannot access '/kaggle/input/cbert-after-preprocessing-by-1': No such file or directory


In [9]:
!wget https://hasocfire.github.io/hasoc/2019/files/english_dataset.zip
!unzip english_dataset.zip
!rm -rf english_dataset.zip __MACOSX/
!mv english_dataset/english_dataset.tsv english_dataset/train.tsv
!mv english_dataset/ dataset
!mv dataset/hasoc2019_en_test-2919.tsv dataset/test.tsv
!pip install emoji
!pip install ekphrasis

import emoji
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons


def reframe_sentence(x):
    annotate_list = ["<hashtag>", "<allcaps>", "<elongated>", "<repeated>",
        '<emphasis>', '<censored>', "</hashtag>", "</allcaps>", "</elongated>", "</repeated>",
        '</emphasis>', '</censored>']

    pre_processed_text_temp = text_processor.pre_process_doc(emoji.demojize(x))
    pre_processed_text = []

    for i in pre_processed_text_temp:
        if i not in annotate_list:
            pre_processed_text.append(i)

    res = " ".join(pre_processed_text)
    return res


text_processor = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,
    segmenter="twitter",
    corrector="twitter",
    unpack_hashtags=True,
    unpack_contractions=True,
    spell_correct_elong=True,
    speel_correction=True,
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts=[emoticons],
    fix_text=True,
    remove_tags=True
)

df = pd.read_csv("dataset/train.tsv", sep='\t')
df['text'].replace('', np.nan, inplace=True)
df['task_1'].replace('', np.nan, inplace=True)
df.dropna(subset=['text', 'task_1'], inplace=True)
df['text']=df['text'].apply(reframe_sentence)
df['humor'] = df['task_1']
df.drop(['text_id', 'task_1', 'task_2', 'task_3'],axis=1,inplace=True)

df.to_csv("train_pre.csv", index=False)

--2020-08-01 08:23:56--  https://hasocfire.github.io/hasoc/2019/files/english_dataset.zip
Resolving hasocfire.github.io (hasocfire.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to hasocfire.github.io (hasocfire.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 585423 (572K) [application/zip]
Saving to: ‘english_dataset.zip’

english_dataset.zip 100%[===================>] 571.70K  --.-KB/s    in 0.06s   

2020-08-01 08:23:57 (9.21 MB/s) - ‘english_dataset.zip’ saved [585423/585423]

Archive:  english_dataset.zip
   creating: english_dataset/
  inflating: __MACOSX/._english_dataset  
  inflating: english_dataset/english_dataset.tsv  
  inflating: __MACOSX/english_dataset/._english_dataset.tsv  
  inflating: english_dataset/hasoc2019_en_test-2919.tsv  
  inflating: __MACOSX/english_dataset/._hasoc2019_en_test-2919.tsv  
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgra

/opt/conda/lib/python3.7/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt


/opt/conda/lib/python3.7/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading twitter - 1grams ...


In [10]:

df = pd.read_csv('train_pre.csv')

def encode(instance):
    if instance=="NOT":
        return 0
    if instance=="HOF":
        return 1

df['humor'] = df['humor'].apply(encode)

df = df.sample(len(df))
df_train = df[:training_sample_count*4]
display(df_train.head(3))

df_test = df[training_sample_count*4:]
display(df_test.head(3))

,text,humor
2148,jamshedpur state doctors stands in solidarity ...,0
2168,murderer american hunter says she has no regre...,0
652,this is eerily similar to how the vietnam war ...,0


,text,humor
5631,<user> you are an evil man . murderer,1
3324,could this be our new prime minister who will ...,0
2704,<user> this is for you shame on icc <url>,0


In [11]:
# df = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/dataset.csv')

# df_train = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/train.csv')
# display(df_train.head(3))
# df_train = df_train[:training_sample_count*5]

# df_test = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/dev.csv')
# display(df_test.head(3))
# df_test = df_test[:test_count]

In [12]:
test_df_y = df_test.copy()
del df_test['humor']

df_sub = test_df_y.copy()

print(len(df),len(df_train),len(df_test))
display(df_train.head())
display(df_test.head())

5852 4800 1052


,text,humor
2148,jamshedpur state doctors stands in solidarity ...,0
2168,murderer american hunter says she has no regre...,0
652,this is eerily similar to how the vietnam war ...,0
921,<user> looking desperately for that attention ...,1
3954,defeat go pin 2020 dictator trump impeachment ...,1


,text
5631,<user> you are an evil man . murderer
3324,could this be our new prime minister who will ...
2704,<user> this is for you shame on icc <url>
5793,baap baap hota hai cockroaches will always be ...
97,ian bagg part <number> : he once told a featur...


In [13]:
print(list(df_train.columns))


['text', 'humor']


In [14]:
output_categories = list(df_train.columns[[1]])
input_categories = list(df_train.columns[[0]])

if HAS_ANS:
    output_categories = list(df_train.columns[11:])
    input_categories = list(df_train.columns[[1,2,5]])
    

TARGET_COUNT = len(output_categories)

print('\ninput categories:\n\t', input_categories)
print('\noutput TARGET_COUNT:\n\t', TARGET_COUNT)
print('\noutput categories:\n\t', output_categories)


input categories:
	 ['text']

output TARGET_COUNT:
	 1

output categories:
	 ['humor']


In [15]:
def _convert_to_transformer_inputs(title, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""
    
    def return_id(str1, str2, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy)
        
        input_ids =  inputs["input_ids"]
        input_masks = [1] * len(input_ids)
        input_segments = inputs["token_type_ids"]
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]
    
    input_ids_q, input_masks_q, input_segments_q = return_id(
        title, None, 'longest_first', max_sequence_length)
    
#     input_ids_a, input_masks_a, input_segments_a = return_id(
#         '', None, 'longest_first', max_sequence_length)
        
    return [input_ids_q, input_masks_q, input_segments_q]

def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    input_ids_q, input_masks_q, input_segments_q = [], [], []
#     input_ids_a, input_masks_a, input_segments_a = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t = instance.text

        ids_q, masks_q, segments_q = \
        _convert_to_transformer_inputs(t, tokenizer, max_sequence_length)
        
        input_ids_q.append(ids_q)
        input_masks_q.append(masks_q)
        input_segments_q.append(segments_q)
#         input_ids_a.append(ids_a)
#         input_masks_a.append(masks_a)
#         input_segments_a.append(segments_a)
        
    return [np.asarray(input_ids_q, dtype=np.int32), 
            np.asarray(input_masks_q, dtype=np.int32), 
            np.asarray(input_segments_q, dtype=np.int32)]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [16]:
outputs = compute_output_arrays(df_train, output_categories)
inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

In [17]:
def compute_spearmanr_ignore_nan(trues, preds):
    rhos = []
    for tcol, pcol in zip(np.transpose(trues), np.transpose(preds)):
        rhos.append(spearmanr(tcol, pcol).correlation)
    return np.nanmean(rhos)

def create_model():
    q_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
#     a_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
#     a_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
#     a_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    config = BertConfig() # print(config) to see settings
    config.output_hidden_states = False # Set to True to obtain hidden states
    # caution: when using e.g. XLNet, XLNetConfig() will automatically use xlnet-large config
    
    bert_model = TFBertModel.from_pretrained('bert-base-uncased', config=config)
    
    # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
    q_embedding = bert_model(q_id, attention_mask=q_mask, token_type_ids=q_atn)[0]
#     a_embedding = bert_model(a_id, attention_mask=a_mask, token_type_ids=a_atn)[0]
    
    q = tf.keras.layers.GlobalAveragePooling1D()(q_embedding)
#     a = tf.keras.layers.GlobalAveragePooling1D()(a_embedding)
    
#     x = tf.keras.layers.Concatenate()([q, q])
    TFBertModel
    x = tf.keras.layers.Dropout(0.2)(q)
    
    x = tf.keras.layers.Dense(TARGET_COUNT, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn], outputs=x)
    
    return model

In [18]:
output_categories

['humor']

In [19]:
# Evaluation Metrics
import sklearn
def print_evaluation_metrics(y_true, y_pred, label='', is_regression=True, label2=''):
    print('==================', label2)
    ### For regression
    if is_regression:
        print('mean_absolute_error',label,':', sklearn.metrics.mean_absolute_error(y_true, y_pred))
        print('mean_squared_error',label,':', sklearn.metrics.mean_squared_error(y_true, y_pred))
        print('r2 score',label,':', sklearn.metrics.r2_score(y_true, y_pred))
        #     print('max_error',label,':', sklearn.metrics.max_error(y_true, y_pred))
        return sklearn.metrics.mean_squared_error(y_true, y_pred)
    else:
        ### FOR Classification
        print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
        print('average_precision_score',label,':', sklearn.metrics.average_precision_score(y_true, y_pred))
        print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
        print('accuracy_score',label,':', sklearn.metrics.accuracy_score(y_true, y_pred))
        print('f1_score',label,':', sklearn.metrics.f1_score(y_true, y_pred))
        
        matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)
        print(matrix)
        TP,TN,FP,FN = matrix[1][1],matrix[0][0],matrix[0][1],matrix[1][0]
        Accuracy = (TP+TN)/(TP+FP+FN+TN)
        Precision = TP/(TP+FP)
        Recall = TP/(TP+FN)
        F1 = 2*(Recall * Precision) / (Recall + Precision)
        print('Acc', Accuracy, 'Prec', Precision, 'Rec', Recall, 'F1',F1)
        return sklearn.metrics.accuracy_score(y_true, y_pred)

print_evaluation_metrics([1,0], [0.9,0.1], '', True)
print_evaluation_metrics([1,0], [1,1], '', False)

mean_absolute_error  : 0.09999999999999999
mean_squared_error  : 0.009999999999999998
r2 score  : 0.96
balanced_accuracy_score  : 0.5
average_precision_score  : 0.5
balanced_accuracy_score  : 0.5
accuracy_score  : 0.5
f1_score  : 0.6666666666666666
[[0 1]
 [0 1]]
Acc 0.5 Prec 0.5 Rec 1.0 F1 0.6666666666666666


0.5

In [20]:
min_acc = 0
min_test = []
valid_preds = []
test_preds = []
best_model = False
best_lr = 1e-5
best_threshold = 0
for LR in [1e-5, 2e-5, 3e-5]:
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    print('LR=', LR)
    gkf = GroupKFold(n_splits=5).split(X=df_train.text, groups=df_train.text)

    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold not in range(running_folds):
            continue
        train_inputs = [(inputs[i][train_idx])[:training_sample_count] for i in range(len(inputs))]
        train_outputs = (outputs[train_idx])[:training_sample_count]

        valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
        valid_outputs = outputs[valid_idx]

        print(np.array(train_inputs).shape, np.array(train_outputs).shape)
#         print(train_idx[:10], valid_idx[:10])

#         K.clear_session()
        model = create_model()
        optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
        model.compile(loss='binary_crossentropy', optimizer=optimizer)
        model.fit(train_inputs, train_outputs, epochs=6, batch_size=16)
        preds = model.predict(valid_inputs)
        valid_preds.append(preds)
        for xx in range(10):
            t = xx/10
            acc = print_evaluation_metrics(np.array(valid_outputs), np.array(valid_preds[-1]>t), 'on #'+str(xx+1), is_regression=False)
            if acc > min_acc:
                print('new acc >> ', acc)
                print('new threshold >> ', t)
                print('LR >> ', LR)
                best_threshold = t
                min_acc = acc
                best_model = model
            print(' ')

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 1e-05
(3, 1200, 256) (1200, 1)



Epoch 1/6
75/75 [==============================] - 36s 473ms/step - loss: 0.6529
Epoch 2/6
75/75 [==============================] - 36s 474ms/step - loss: 0.6050
Epoch 3/6
75/75 [==============================] - 36s 474ms/step - loss: 0.5463
Epoch 4/6
75/75 [==============================] - 35s 473ms/step - loss: 0.4622
Epoch 5/6
75/75 [==============================] - 36s 474ms/step - loss: 0.3304
Epoch 6/6
75/75 [==============================] - 35s 473ms/step - loss: 0.1671
balanced_accuracy_score on #1 : 0.5
average_precision_score on #1 : 0.40729166666666666
balanced_accuracy_score on #1 : 0.5
accuracy_score on #1 : 0.40729166666666666
f1_score on #1 : 0.5788304959289415
[[  0 569]
 [  0 391]]
Acc 0.40729166666666666 Prec 0.40729166666666666 Rec 1.0 F1 0.5788304959289415
new acc >>  0.40729166666666666
new threshold >>  0.0
LR >>  1e-05
 
balanced_accuracy_score on #2 : 0.5994071350554434
average_precision_score on #2 : 0.46366520239335673
balanced_accuracy_score on #2 : 0.59

In [21]:
best_model.save_weights('/kaggle/working/best_model_humour.h5')

In [22]:
print('best acc >> ', min_acc)
print('threshold >> ', best_threshold)
print('LR >> ', best_lr)

best acc >>  0.6614583333333334
threshold >>  0.8
LR >>  1e-05


In [23]:
!ls /kaggle/working

__notebook__.ipynb  best_model_humour.h5  dataset  train_pre.csv  wandb


In [24]:
best_model = create_model().load_weights('best_model_humour.h5')

In [25]:
len(valid_inputs[0])

960

In [26]:
print(valid_outputs.shape, valid_preds[-1].shape)
print_evaluation_metrics(np.array(valid_outputs), np.array(valid_preds[-1]), '')

(960, 1) (960, 1)
mean_absolute_error  : 0.36005773315693024
mean_squared_error  : 0.2916322569774079
r2 score  : -0.2080613812107166


0.2916322569774079

In [27]:
# %%time
min_test = best_model.predict(test_inputs)

## use min_test
min_test

AttributeError: 'NoneType' object has no attribute 'predict'

In [28]:
df_test.to_csv('df_test.csv')
test_df_y.to_csv('test_df_y.csv')

In [29]:
df_sub = test_df_y.copy()
# df_sub['pred'] = np.average(test_preds, axis=0) # for weighted average set weights=[...]
df_sub['pred'] = min_test


print_evaluation_metrics(df_sub['humor'], df_sub['pred'], '', True)


df_sub.to_csv('sub1.csv', index=False)
df_sub.head()

ValueError: Length of values does not match length of index